# Training a classifier

### Data
Generally, when we have to deal with image, text, audio or video data, we can use standard python packages that load data into a numpy array. Then we can convert this array into a __torch.*Tensor__.

* For images, packages such as Pillow, OpenCV are useful
* For audio, packages such as scipy and librosa
* For text, either raw Python or Cython based loading, or NLTK and SpaCy are useful

Specifically for vision, we have created a package called __torchvision__, that has data loaders for common datasets such as Imagenet, CIFAR10, MNIST, etc. and data transformers for images, viz., __torchvision.datasets__ and __torch.utils.data.DataLoader__.

For this tutorial, we will use the __CIFAR10__ dataset. It has the classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. The images in CIFAR-10 are of size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.

### Training an image classifier
1. Load and normalizing the CIFAR10 training and test datasets using __torchvision__
1. Define a Convolutional Neural Network
1. Define a loss function
1. Train the network on the training data
1. Test the network on the test data

#### 1. Loading and normalizing CIFAR10

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms